<a href="https://colab.research.google.com/github/padmakar-rp/VADER-Sentiment-Chatbot/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.sentiment import SentimentIntensityAnalyzer
import string
import random
import pandas as pd

# ==============================================================================
# 1. SETUP AND INITIALIZATION
# ==============================================================================

# Download required NLTK data
# 'punkt' is for splitting sentences/words
# 'stopwords' allows us to filter out common words like "the", "is", "at"
# 'vader_lexicon' is the dictionary used for sentiment scoring
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('vader_lexicon', quiet=True)
nltk.download('punkt_tab', quiet=True) # Added to fix LookupError

# Initialize the NLP tools
sia = SentimentIntensityAnalyzer() # VADER analyzer (best for social media/chat text)
stemmer = PorterStemmer()          # Reduces words to their root (e.g., "running" -> "run")
stop_words = set(stopwords.words('english')) # Set of common English filler words

# ==============================================================================
# 2. HELPER FUNCTIONS
# ==============================================================================

def preprocess_text(text):
    """
    Cleans the user input for logging purposes.
    1. Lowercases text.
    2. Removes punctuation and stop words.
    3. Stems words to their root form.
    """
    text = text.lower()
    tokens = word_tokenize(text)
    # Remove punctuation
    tokens = [token for token in tokens if token not in string.punctuation]
    # Remove stop words (common words that don't add much meaning)
    tokens = [token for token in tokens if token not in stop_words]
    # Apply stemming (reduce words to root form)
    tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(tokens)

def analyze_sentiment(text):
    """
    Analyzes the sentiment of the text using VADER.
    Returns the sentiment label and the specific score.
    """
    # We use the raw 'text' here because VADER handles capitalization and punctuation
    # (like "GREAT!") better than processed text.
    scores = sia.polarity_scores(text)

    # The 'compound' score ranges from -1 (Extremely Negative) to +1 (Extremely Positive)
    compound = scores['compound']

    # Determine label based on standard VADER thresholds
    if compound >= 0.05:
        sentiment = 'Positive'
    elif compound <= -0.05:
        sentiment = 'Negative'
    else:
        sentiment = 'Neutral'

    return sentiment, scores

def get_chatbot_response(sentiment):
    """
    Selects a random response based on the detected sentiment.
    """
    # Expanded dictionary with 20 responses per category
    responses = {
        'Positive': [
            "That's wonderful! I'm glad to hear that.",
            "Your positive energy is great!",
            "Excellent! Keep that optimism going!",
            "That sounds amazing!",
            "I'm thrilled for you!",
            "It's great to see you in such a good mood.",
            "That just made my day!",
            "Keep up the fantastic work!",
            "Positive vibes only! Love it.",
            "That is genuinely good news.",
            "You seem very happy, and that makes me happy!",
            "Awesome! Tell me more about it.",
            "Whatever you're doing, it's working!",
            "That's the spirit!",
            "Sending you a virtual high-five!",
            "I love hearing success stories like this.",
            "Your enthusiasm is contagious.",
            "Brilliant! Let's keep this momentum.",
            "That is music to my ears.",
            "So happy that things are going well for you."
        ],
        'Negative': [
            "I'm sorry to hear that. How can I help?",
            "That sounds challenging. Tell me more.",
            "I understand your concerns. Let's discuss this.",
            "I'm here for you if you need to vent.",
            "That sounds rough. Take your time.",
            "It's okay to feel this way sometimes.",
            "Is there anything specific bothering you?",
            "I wish I could do more to help.",
            "Sending you some virtual support.",
            "Let's try to work through this together.",
            "I'm listening. Please go on.",
            "That must be very difficult.",
            "Don't be too hard on yourself.",
            "Take a deep breath. We can figure this out.",
            "I'm sorry you're having a bad time.",
            "Is there a solution we can look for?",
            "That sounds frustrating.",
            "My virtual heart goes out to you.",
            "Let it all out, I'm just a bot but I'm listening.",
            "Tomorrow is a new day, but for now, I hear you."
        ],
        'Neutral': [
            "Interesting point. Tell me more.",
            "I see. Could you provide more details?",
            "Understood. What else would you like to discuss?",
            "Okay, noted.",
            "I'm following you. Go on.",
            "That is a fair observation.",
            "Could you clarify what you mean?",
            "I am processing that information.",
            "Got it. Anything else?",
            "That is one way to look at it.",
            "Let's explore that topic further.",
            "I'm neutral on that as well.",
            "Do you have any other thoughts on this?",
            "Straightforward and to the point.",
            "I hear what you are saying.",
            "Can you elaborate on that?",
            "Okay, let's switch gears a bit.",
            "I'm ready for your next input.",
            "That seems like a balanced view.",
            "Right. Moving on..."
        ]
    }
    return random.choice(responses[sentiment])

# ==============================================================================
# 3. MAIN EXECUTION LOOP
# ==============================================================================

conversation_log = []

print("=" * 60)
print("SENTIMENT ANALYSIS CHATBOT (VADER ENGINE)")
print("=" * 60)
print("Chat with the bot (type 'exit' to quit)\n")

while True:
    # Get user input and strip whitespace
    user_input = input("You: ").strip()

    # Check for exit condition
    if user_input.lower() == 'exit':
        print("Bot: Thank you for chatting! Goodbye!")
        break

    # Skip empty inputs
    if not user_input:
        continue

    # --- Processing Steps ---

    # 1. Preprocess text (mostly for our records/logging)
    processed_text = preprocess_text(user_input)

    # 2. Analyze sentiment (using the raw input for better accuracy)
    sentiment, scores = analyze_sentiment(user_input)

    # 3. Generate a random response based on the category
    response = get_chatbot_response(sentiment)

    # 4. Log the details of this interaction
    conversation_log.append({
        'User Input': user_input,
        'Processed Text': processed_text,
        'Sentiment': sentiment,
        'Compound Score': round(scores['compound'], 3),
        'Bot Response': response
    })

    # 5. Display the response and the score to the user
    print(f"Bot: {response}")
    print(f"[Sentiment: {sentiment}, Score: {scores['compound']:.3f}]\n")
    print("-" * 60)

# ==============================================================================
# 4. DATA ANALYSIS AND EXPORT
# ==============================================================================

print("\n" + "=" * 60)
print("FINAL CONVERSATION LOG")
print("=" * 60)

# Create a Pandas DataFrame to display the conversation history neatly
df = pd.DataFrame(conversation_log)

if not df.empty:
    print(df.to_string(index=False))

    # Calculate simple statistics
    print("\n--- Statistics ---")
    print(f"Total Messages: {len(conversation_log)}")
    print(f"Positive Messages: {len([x for x in conversation_log if x['Sentiment'] == 'Positive'])}")
    print(f"Negative Messages: {len([x for x in conversation_log if x['Sentiment'] == 'Negative'])}")
    print(f"Neutral Messages:  {len([x for x in conversation_log if x['Sentiment'] == 'Neutral'])}")
else:
    print("No conversation data to report.")

SENTIMENT ANALYSIS CHATBOT (VADER ENGINE)
Chat with the bot (type 'exit' to quit)

You: hi
Bot: I'm following you. Go on.
[Sentiment: Neutral, Score: 0.000]

------------------------------------------------------------
You: i am very sad 
Bot: I wish I could do more to help.
[Sentiment: Negative, Score: -0.526]

------------------------------------------------------------
You: iam do great at my workshop
Bot: It's great to see you in such a good mood.
[Sentiment: Positive, Score: 0.625]

------------------------------------------------------------
You: can you tell me how the code works 
Bot: I hear what you are saying.
[Sentiment: Neutral, Score: 0.000]

------------------------------------------------------------
You: exit
Bot: Thank you for chatting! Goodbye!

FINAL CONVERSATION LOG
                        User Input     Processed Text Sentiment  Compound Score                               Bot Response
                                hi                 hi   Neutral           0.000 